# ML-Fundamentals - Neural Networks - Exercise: Neural Network Framework

# Tasks
Your main goal is to extend the existing framework, to perform experiments with different model combinations and to document your observations. Here is a list of necessary tasks and some ideas for additional points:
  * (6) Write a data loader for a different image dataset, e.g., CIFAR or Labelled Faces in the Wild. Feel free to search a dataset you like to classify. Create and train a simple fully connected network on that dataset in this notebook.
  * (10) Implement the `Conv` and `Pool` Layer in `layer.py`. Create and train a convolutional neural network on Mnist and your chosen dataset in this notebook.

Bonus points
  * (5) 1 to 5 points are given for improving the class and method comments in the framework files. Points are given based on the quality and quantity of the comments.
  * (1) For each additional implemented activation functions in `activation_func.py` you get 1 bonus point (max 4 points). Test your implementation in this notebook and observe effects on your networks. Keep an eye on your layer initialization.
  * (2) Implement `Dropout` in `layer.py` and test your implementation with a toy example. Create and train a model that includes Dropout as a layer.
  * (5) Implement `Batchnorm` in `layer.py` and test your implementation with a toy example. Create and train a model that includes Dropout as a layer.
  * (4) Implement another optimization algorithm in `optimizer.py`. Train one of your models with that new optimizer.
  * (5) Do something extra, up to 5 points.  
  
Please document thoroughly and explain what you do in your experiments, so that work in the notebook is comprehensible, else no points are given.

Fragen
* Sollen wir nur für Pool Layer nur Max forward und backward implementieren? Falls nein, wie solen wir das mit der backward implementierung machen?
* Siehe in layers.py: 
    * In Pooling.backward: What is if not only one pixel has the max value?
    * In Conv Layer: soll zu jedem Pixel bias hinzugefügt werden oder erst nach sum?
    * In Dropout: maske für alle images gleich oder für jedes eine andere? Und wie sollen wir damit umgehen, wenn wir vorhersagen wollen?
    * Braucht man nach Dropout oder Pool Layer eine Aktivierungsfunktion?
    * Pooling Layer gibt immer gleichen loss zurück, woran kann das liegen?

# Requirements

## Python-Modules

In [ ]:
%load_ext autoreload
%autoreload 2
# custom 
from htw_nn_framework.networks import NeuralNetwork
from htw_nn_framework.layer import *
from htw_nn_framework.activation_func import *
from htw_nn_framework.loss_func import *
from htw_nn_framework.optimizer import *
from htw_nn_framework.cifar import *

# third party
from deep_teaching_commons.data.fundamentals.mnist import Mnist

## Data

In [ ]:
# create mnist loader from deep_teaching_commons
mnist_loader = Mnist(data_dir='data')

# load all data, labels are one-hot-encoded, images are flatten and pixel squashed between [0,1]
train_images, train_labels, test_images, test_labels = mnist_loader.get_all_data(flatten=False, one_hot_enc=False, normalized=True)
print(train_images.shape, train_labels.shape)

# reshape to match generell framework architecture 
train_images, test_images = train_images.reshape(60000, 1, 28, 28), test_images.reshape(10000, 1, 28, 28)            
print(train_images.shape, train_labels.shape)

# shuffle training data
shuffle_index = np.random.permutation(60000)
train_images, train_labels = train_images[shuffle_index], train_labels[shuffle_index]

# MNIST Fully Connected Network Example
This model and optimization is taken from `framework_exercise.ipynb` as an example for a typical pipeline using the framework files.

In [ ]:
train_images_batch, train_labels_batch, test_images_batch, test_labels_batch = train_images[:100,:,:,:], train_labels[:100], test_images[:100,:,:,:], test_labels[:100]

In [ ]:
# Design a three hidden layer architecture with dense layer
# and ReLU as activation function
def fcn_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(784, 500)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(500, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn = NeuralNetwork(fcn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn = Optimizer.sgd(fcn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

# Todo: Your Extensions and Experiments
# MNIST
## Convolutional Neural Network

In [ ]:
# Design a one hidden layer architecture with conv layer
# and ReLU as activation function
def cnn_mnist():
    hidden_01 = Conv(train_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    flat = Flatten()
    hidden_02 = FullyConnected(784, 1500)
    relu_02 = ReLU()
    ouput = FullyConnected(1500, 10)
    return [hidden_01, relu_01, flat, hidden_02, relu_02, ouput]

# create a neural network on specified architecture with softmax as score function
cnn = NeuralNetwork(cnn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn = Optimizer.sgd(cnn, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

## Convolutional Neural Network with Pooling

In [ ]:
# Design a one hidden layer architecture with conv layer, pooling layer
# and ReLU as activation function
def cnn_pool_mnist():
    hidden_01 = Conv(train_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    pool = Pool(train_images_batch.shape, np.max, 2, 1)
    flat = Flatten()
    ouput = FullyConnected(729, 10)
    return [hidden_01, relu_01, pool, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn_pool = NeuralNetwork(cnn_pool_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn_pool = Optimizer.sgd(cnn_pool, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=20, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

## Fully Conected Neural Network with Dropout

In [ ]:
# Design a two hidden layer architecture with dense layer, dropout layer
# and ReLU as activation function
def fcn_dropout_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(784, 500)
    relu_01 = ReLU()
    dropout = Dropout()
    hidden_02 = FullyConnected(500, 100)
    relu_02 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, dropout, hidden_02, relu_02, ouput]

# create a neural network on specified architecture with softmax as score function
fcn_dropout = NeuralNetwork(fcn_dropout_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn_dropout = Optimizer.sgd(fcn_dropout, train_images_batch, train_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=20, learning_rate=0.01, X_test=test_images_batch, y_test=test_labels_batch, verbose=True)

# CIFAR10

In [ ]:
Cifar.load()

In [ ]:
tr_images, tr_labels, te_images, te_labels = Cifar.get()

In [ ]:
# shuffle training data
shuffle_index = np.random.permutation(len(tr_images))
tr_images, tr_labels = tr_images[shuffle_index], tr_labels[shuffle_index]

tr_images_batch, tr_labels_batch, te_images_batch, te_labels_batch = tr_images[:100,:,:,:], tr_labels[:100], te_images[:100,:,:,:], te_labels[:100]

## Fully Connected Network

In [ ]:
# Design a three hidden layer architecture with dense layer
# and ReLU as activation function
def fcn_cifar():
    flat = Flatten()
    hidden_01 = FullyConnected(1024, 500)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(500, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn_c10 = NeuralNetwork(fcn_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn_c10 = Optimizer.sgd(fcn_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=32, epoch=20, learning_rate=0.01, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)

## Convolutional Neural Network

In [ ]:
# Design a one hidden layer architecture with conv layer
# and ReLU as activation function
def cnn_cifar():
    conv = Conv(tr_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    flat = Flatten()
    ouput = FullyConnected(1024, 10)
    return [conv, relu_01, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn_c10 = NeuralNetwork(cnn_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn_c10 = Optimizer.sgd(cnn_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=32, epoch=20, learning_rate=0.01, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)

## Convolutional Neural Network with Pooling

In [ ]:
def cnn_pool_cifar():
    conv = Conv(tr_images_batch.shape, 1, 3, 1, True)
    relu_01 = ReLU()
    pool = Pool(tr_images_batch.shape, np.max, 2, 1)
    flat = Flatten()
    ouput = FullyConnected(961, 10)
    return [conv, relu_01, pool, flat, ouput]

# create a neural network on specified architecture with softmax as score function
cnn_pool_c10 = NeuralNetwork(cnn_pool_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
cnn_c10 = Optimizer.sgd(cnn_pool_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=20, learning_rate=0.01, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)

## Fully Conected Neural Network with Dropout

In [ ]:
# Design a two hidden layer architecture with dense layer, dropout layer
# and ReLU as activation function
def fcn_dropout_cifar():
    flat = Flatten()
    hidden_01 = FullyConnected(1024, 500)
    relu_01 = ReLU()
    dropout = Dropout()
    hidden_02 = FullyConnected(500, 100)
    relu_02 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, dropout, hidden_02, relu_02, ouput]

# create a neural network on specified architecture with softmax as score function
fcn_dropout_c10 = NeuralNetwork(fcn_dropout_cifar(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn_dropout_c10 = Optimizer.sgd(fcn_dropout_c10, tr_images_batch, tr_labels_batch, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=20, learning_rate=0.01, X_test=te_images_batch, y_test=te_labels_batch, verbose=True)